In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm, ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# ----------------------------------------------------------------------------
# Assume you already have:
# texas_counties_stats_gdf with columns:
#   'Pre_Event_mean', 'Event_mean', 'Difference', 'PovRate', 'geometry'
#
# Also assume it's in EPSG:4326 (lat/lon).
# ----------------------------------------------------------------------------

# Let's define some city locations to mark on the maps
cities = {
    'Houston':  (-95.3698, 29.7604),
    'Dallas':   (-96.7970, 32.7767)
}

# Helper function to plot city markers on a given Axes
def plot_city_markers(ax, city_dict, text_color='black'):
    for name, (lon, lat) in city_dict.items():
        ax.plot(lon, lat, marker='o', color='black', markersize=5)
        # Offset the text a bit so it doesn't overlap the marker
        ax.text(lon + 0.3, lat + 0.3, name, color=text_color, fontsize=10)

# ----------------------------------------------------------------------------
# 1. A 2x2 Subplot: (1) Pre-Event Radiance, (2) Event Radiance,
#    (3) Difference, (4) Poverty Rate
#    Each with discrete color intervals & markers for Houston/Dallas
# ----------------------------------------------------------------------------
def plot_four_panel_maps(gdf):
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    ax1, ax2, ax3, ax4 = axes.flat

    # --------------------------------------------------
    # (A) Pre-Event Radiance
    # --------------------------------------------------
    # Define bins & colormap
    bins_pre = np.linspace(0, 30, 7)  # Example: 0..30 in 5-6 intervals
    cmap_pre = plt.get_cmap('plasma', len(bins_pre) - 1)
    norm_pre = BoundaryNorm(bins_pre, ncolors=cmap_pre.N)

    im1 = gdf.plot(
        column='Pre_Event_mean',
        cmap=cmap_pre,
        norm=norm_pre,
        linewidth=0.5,
        ax=ax1,
        edgecolor='0.7'
    )
    ax1.set_title("Pre-Event Radiance", fontsize=14)
    ax1.axis('off')
    # Add city markers
    plot_city_markers(ax1, cities, text_color='white')

    # Colorbar
    cbar1 = fig.colorbar(
        plt.cm.ScalarMappable(norm=norm_pre, cmap=cmap_pre),
        ax=ax1,
        orientation='horizontal',
        fraction=0.046,
        pad=0.08,
        ticks=bins_pre
    )
    cbar1.set_label("Radiance (W/m²·sr·µm)", fontsize=11)

    # --------------------------------------------------
    # (B) Event Radiance
    # --------------------------------------------------
    bins_event = np.linspace(0, 30, 7)  # Adjust as needed
    cmap_event = plt.get_cmap('plasma', len(bins_event) - 1)
    norm_event = BoundaryNorm(bins_event, ncolors=cmap_event.N)

    im2 = gdf.plot(
        column='Event_mean',
        cmap=cmap_event,
        norm=norm_event,
        linewidth=0.5,
        ax=ax2,
        edgecolor='0.7'
    )
    ax2.set_title("Event Radiance", fontsize=14)
    ax2.axis('off')
    plot_city_markers(ax2, cities, text_color='white')

    cbar2 = fig.colorbar(
        plt.cm.ScalarMappable(norm=norm_event, cmap=cmap_event),
        ax=ax2,
        orientation='horizontal',
        fraction=0.046,
        pad=0.08,
        ticks=bins_event
    )
    cbar2.set_label("Radiance (W/m²·sr·µm)", fontsize=11)

    # --------------------------------------------------
    # (C) Difference (Event - PreEvent)
    # --------------------------------------------------
    # Example: -5..5 in increments
    bins_diff = np.arange(-5, 6, 2)  # -5, -3, -1, 1, 3, 5
    cmap_diff = plt.get_cmap('coolwarm', len(bins_diff) - 1)
    norm_diff = BoundaryNorm(bins_diff, ncolors=cmap_diff.N)

    im3 = gdf.plot(
        column='Difference',
        cmap=cmap_diff,
        norm=norm_diff,
        linewidth=0.5,
        ax=ax3,
        edgecolor='0.7'
    )
    ax3.set_title("Difference (Event - PreEvent)", fontsize=14)
    ax3.axis('off')
    plot_city_markers(ax3, cities, text_color='black')

    cbar3 = fig.colorbar(
        plt.cm.ScalarMappable(norm=norm_diff, cmap=cmap_diff),
        ax=ax3,
        orientation='horizontal',
        fraction=0.046,
        pad=0.08,
        ticks=bins_diff
    )
    cbar3.set_label("Radiance Difference", fontsize=11)

    # --------------------------------------------------
    # (D) Poverty Rate
    # --------------------------------------------------
    # Suppose range is from 5% to 35% with intervals of 5
    bins_poverty = np.arange(5, 40, 5)  # 5..35
    cmap_pov = plt.get_cmap('OrRd', len(bins_poverty) - 1)
    norm_pov = BoundaryNorm(bins_poverty, ncolors=cmap_pov.N)

    im4 = gdf.plot(
        column='PovRate',
        cmap=cmap_pov,
        norm=norm_pov,
        linewidth=0.5,
        ax=ax4,
        edgecolor='0.7'
    )
    ax4.set_title("Poverty Rate (%)", fontsize=14)
    ax4.axis('off')
    plot_city_markers(ax4, cities, text_color='black')

    cbar4 = fig.colorbar(
        plt.cm.ScalarMappable(norm=norm_pov, cmap=cmap_pov),
        ax=ax4,
        orientation='horizontal',
        fraction=0.046,
        pad=0.08,
        ticks=bins_poverty
    )
    cbar4.set_label("Poverty Rate (%)", fontsize=11)

    fig.suptitle("Texas Counties: Pre-Event, Event, Difference, & Poverty", fontsize=16)
    plt.tight_layout()
    plt.show()


# ----------------------------------------------------------------------------
# 2. Bivariate Choropleth: Combining 'Difference' & 'PovRate'
# ----------------------------------------------------------------------------
# We'll divide each variable into 3 classes (low, medium, high).
def plot_bivariate_map(gdf):
    # Step 1: Define bins for difference & poverty
    diff_bins = [-5, -1, 1, 5]   # example: < -1 (low), between -1 and 1 (mid), > 1 (high)
    pov_bins  = [0, 15, 25, 100] # example: <15% (low), 15-25% (med), >25% (high)

    # Classify difference & poverty
    gdf['diff_class'] = pd.cut(gdf['Difference'], bins=diff_bins, labels=['LowDiff','MidDiff','HighDiff'])
    gdf['pov_class']  = pd.cut(gdf['PovRate'], bins=pov_bins, labels=['LowPov','MidPov','HighPov'])

    # Combine into a single bivariate category
    gdf['bivar_cat'] = gdf['diff_class'].astype(str) + "_" + gdf['pov_class'].astype(str)

    # Step 2: Define a color scheme for the 3x3 combos
    # For instance, rows = difference, cols = poverty
    # We'll manually map each combination to a color
    bivar_colors = {
        'LowDiff_LowPov':   '#fee8c8',  # pick a lighter color
        'MidDiff_LowPov':   '#fdbb84',
        'HighDiff_LowPov':  '#e34a33',
        'LowDiff_MidPov':   '#d9f0a3',
        'MidDiff_MidPov':   '#addd8e',
        'HighDiff_MidPov':  '#31a354',
        'LowDiff_HighPov':  '#c2e5ff',
        'MidDiff_HighPov':  '#99ccff',
        'HighDiff_HighPov': '#3399ff'
    }

    # Create a fallback color for missing or out-of-range combos
    def get_bivar_color(cat):
        return bivar_colors.get(cat, "#999999")

    # Step 3: Plot the bivariate map
    fig, ax = plt.subplots(figsize=(10,8))
    gdf['bivar_color'] = gdf['bivar_cat'].apply(get_bivar_color)

    gdf.plot(
        color=gdf['bivar_color'],
        linewidth=0.5,
        edgecolor='0.6',
        ax=ax
    )

    ax.set_title("Bivariate Choropleth: Nightlight Difference vs. Poverty Rate", fontsize=14)
    ax.axis('off')
    plot_city_markers(ax, cities, text_color='black')

    plt.tight_layout()
    plt.show()

    # (Optional) You can create a custom legend square showing each 3×3 category
    # but that's a bit more manual work. 


# ----------------------------------------------------------------------------
# 3. Inset Map Example: Zoom in on Houston
# ----------------------------------------------------------------------------
def plot_inset_map(gdf):
    # We'll do a normal map of Difference for the whole state,
    # then create an inset axis that zooms around Houston area.

    fig, ax = plt.subplots(figsize=(10, 8))

    # 3a: Main map for entire Texas
    bins_diff = np.arange(-5, 6, 2)  # same as before
    cmap_diff = plt.get_cmap('coolwarm', len(bins_diff) - 1)
    norm_diff = BoundaryNorm(bins_diff, ncolors=cmap_diff.N)

    gdf.plot(
        column='Difference',
        cmap=cmap_diff,
        norm=norm_diff,
        linewidth=0.5,
        edgecolor='0.7',
        ax=ax
    )
    ax.set_title("Difference (Event - PreEvent) with Houston Inset", fontsize=14)
    ax.axis('off')
    plot_city_markers(ax, cities, text_color='black')

    # Colorbar on main map
    cbar = fig.colorbar(
        plt.cm.ScalarMappable(norm=norm_diff, cmap=cmap_diff),
        ax=ax,
        orientation='horizontal',
        fraction=0.046,
        pad=0.08,
        ticks=bins_diff
    )
    cbar.set_label("Radiance Difference", fontsize=11)

    # 3b: Inset for Houston
    # Create an inset axes on top of 'ax'
    axins = inset_axes(ax,
                       width="45%",  # % of parent_axes width
                       height="45%", # % of parent_axes height
                       loc='lower left',
                       bbox_to_anchor=(0.05, 0.05, 0.4, 0.4),
                       bbox_transform=ax.transAxes,
                       borderpad=2
                      )

    # We can define a bounding box around Houston (roughly).
    # For instance, let's say:
    #   lat range ~ (29, 30)
    #   lon range ~ (-96.5, -94.5)
    # Adjust as needed to see the desired region
    axins.set_xlim([-96.0, -94.6])
    axins.set_ylim([29.3, 30.2])

    # Redraw the same polygons in the inset
    gdf.plot(
        column='Difference',
        cmap=cmap_diff,
        norm=norm_diff,
        linewidth=0.5,
        edgecolor='0.7',
        ax=axins
    )

    # Add the city marker for Houston in the inset
    axins.plot(cities['Houston'][0], cities['Houston'][1], 'ko', markersize=5)
    axins.text(cities['Houston'][0] + 0.05, cities['Houston'][1] + 0.05, "Houston", color='black', fontsize=9)

    axins.axis('off')

    plt.tight_layout()
    plt.show()


# ----------------------------------------------------------------------------
# MAIN EXECUTION
# ----------------------------------------------------------------------------
if __name__ == "__main__":

    # 1. Four-panel figure
    plot_four_panel_maps(texas_counties_stats_gdf)

    # 2. Bivariate map
    plot_bivariate_map(texas_counties_stats_gdf)

    # 3. Inset map focusing on Houston
    plot_inset_map(texas_counties_stats_gdf)
